Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table

In [14]:
# подгружаем .env
load_dotenv()

True

In [15]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [16]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [23]:
# Укажем таблицу для удаления
TABLE_NAME = 'alt_wb_statistic_dag'

In [18]:
# Функция удаления таблицы с помощью sqlalchemy
def delete_table(table_name, conn):
    metadata = MetaData(bind=conn)
    table = Table(table_name, metadata, autoload=True)
    table.drop(conn)

In [24]:
# Удалим таблицу
delete_table(TABLE_NAME, dst_conn)

In [28]:
# Проверим, что таблицы больше нет
pd.read_sql(f'select * from {TABLE_NAME}', dst_conn)

,id,project_id_alt,countryname_off,countryname,countrycode,sector1,year,year_close,totalamt,vvp,population,electricity,rural,target
0,1,P163962,Democratic Republic of the Congo,None,COD,None,2018,2023,200000000.0,NaN,NaN,NaN,NaN,1
1,2,P167672,People's Republic of Bangladesh,None,BGD,None,2018,,58330000.0,NaN,NaN,NaN,NaN,0
2,3,P158768,Islamic Republic of Afghanistan,None,AFG,None,2018,2023,20000000.0,NaN,NaN,NaN,NaN,1
3,4,P161364,Federal Republic of Nigeria,None,NGA,Social Protection,2018,2023,100000000.0,NaN,NaN,NaN,NaN,1
4,5,P161483,Republic of Tunisia,None,TUN,None,2018,2019,500000000.0,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,12996,P003035,Democratic Republic of the Congo,"Congo, Dem. Rep.",COD,Financial sector development,1982,1988,21500000.0,2.160640e+09,27717337.0,0.01,72.417,1
12996,12997,P005865,Republic of Yemen,"Yemen, Rep.",YEM,Fisheries,1982,1991,6000000.0,8.269690e+10,8764621.0,0.01,82.749,1
12997,12998,P007431,Jamaica,Jamaica,JAM,Thermal,1982,1987,30500000.0,3.293533e+09,2235327.0,0.01,52.187,1
12998,12999,P009785,Republic of India,India,IND,Irrigation and Drainage,1982,1989,80300000.0,1.980377e+11,729868013.0,0.01,76.353,1


In [10]:
# Закроем соединения в конце работы
src_conn.dispose()
dst_conn.dispose()